**Домашнее задание**

Возьмите датасет https://www.kaggle.com/ajayrana/hymenoptera-data

Реализуйте сверточную нейронную сеть с использованием tf.keras или keras. Используйте сверточные, пуллинговые и полносвязанные слои. Обучите на train выборке в течении 10 эпох, оцените качество на отложенной выборке

Добавьте в предыдущую архитектуру слои BatchNorm. Обучите на train выборке в течении 10 эпох, оцените качество на отложенной выборке

Создайте модель ResNet 50 (https://www.tensorflow.org/api_docs/python/tf/keras/applications/ResNet50), инициализированную случайными весами, и обучите ее на train выборке в течении 10 эпох, оцените качество на отложенной выборке

Создайте модель ResNet 50, инициализированную весами ImageNet, и обучите ее на train выборке в течении 10 эпох, оцените качество на отложенной выборке

Результат пришлите в виде Jupyter Notebook на github’е или расшаренного Google Colab-блокнота

In [1]:
import numpy as np
from scipy import stats
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Загрузим изображения, приведем к единому размеру и разделим выборки.

In [2]:
shape=(224, 224, 3)
batch_size=32
seed=42

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.15)
val_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)

train = train_generator.flow_from_directory("hymenoptera_data/train", target_size=shape[:2], batch_size=batch_size, class_mode='binary', subset='training', seed=seed)
test = train_generator.flow_from_directory("hymenoptera_data/train", target_size=shape[:2], batch_size=batch_size, class_mode='binary', subset='validation', seed=seed)
val = val_generator.flow_from_directory("hymenoptera_data/val", target_size=shape[:2], batch_size=batch_size, class_mode='binary')

x_val = np.concatenate([val.next()[0] for i in range(val.__len__())])
y_val = np.concatenate([val.next()[1] for i in range(val.__len__())])

Found 208 images belonging to 2 classes.
Found 36 images belonging to 2 classes.
Found 153 images belonging to 2 classes.


Построим простую модель сверточной сети  и обучим ее.

In [3]:
s_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=6,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu',
                           input_shape=shape),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    tf.keras.layers.Conv2D(filters=16, 
                           kernel_size=(5, 5),
                           padding='same',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')]
)

s_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
s_model.summary()
s_model.fit(train, validation_data=test, epochs=10);

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 6)       168       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 6)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 16)      2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                1605664   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3

Посчитаем accuracy на отложенной выборке.

In [4]:
def score_and_show(model, x, y):
    y_pred = model.predict(x).round()
    print(f'Accuracy: {accuracy_score(y, y_pred)}')
    print('\nClassification Report')
    print(classification_report(y, y_pred))
    print('Confusion matrix')
    print(confusion_matrix(y, y_pred))


score_and_show(s_model, x_val, y_val)

Accuracy: 0.4444444444444444

Classification Report
              precision    recall  f1-score   support

         0.0       0.42      0.54      0.47        70
         1.0       0.48      0.36      0.41        83

    accuracy                           0.44       153
   macro avg       0.45      0.45      0.44       153
weighted avg       0.45      0.44      0.44       153

Confusion matrix
[[38 32]
 [53 30]]


Далеко не лучший результат на отложенной выборке.

Построим модель со слоями батч-нормализации и обучим ее.

In [5]:
b_model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=shape),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=6,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=16, 
                           kernel_size=(5, 5),
                           padding='same',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')]
)

b_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
b_model.summary()
b_model.fit(train, validation_data=test, epochs=10);

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 6)       168       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 112, 112, 6)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 6)       24        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 16)      2416      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 56, 56, 16)       

Посчитаем accuracy на отложенной выборке.

In [6]:
score_and_show(b_model, x_val, y_val)

Accuracy: 0.43790849673202614

Classification Report
              precision    recall  f1-score   support

         0.0       0.44      0.77      0.56        70
         1.0       0.45      0.16      0.23        83

    accuracy                           0.44       153
   macro avg       0.44      0.46      0.39       153
weighted avg       0.44      0.44      0.38       153

Confusion matrix
[[54 16]
 [70 13]]


Как видим, введение батч-нормализации в нашем случае не помогло. Модель явно переобучилась и на отложенной выборке показала более худший результат.

Построим модель ResNet 50 инициализированную случайными весами и обучим ее.

In [7]:
i = tf.keras.layers.Input(shape, dtype=tf.uint8)
o = tf.cast(i, tf.float32)
o = tf.keras.applications.resnet50.preprocess_input(o)
o = tf.keras.applications.ResNet50(include_top=False, weights=None)(o)
o = tf.keras.layers.Flatten()(o)
o = tf.keras.layers.Dense(1, activation='sigmoid')(o)

rnnw_model = tf.keras.Model(inputs=[i], outputs=[o])
rnnw_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rnnw_model.summary()
rnnw_model.fit(train, validation_data=test, epochs=10);

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.cast (TFOpLambda)         (None, 224, 224, 3)       0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 100353

Посчитаем accuracy на отложенной выборке.

In [8]:
score_and_show(rnnw_model, x_val, y_val)

Accuracy: 0.45751633986928103

Classification Report
              precision    recall  f1-score   support

         0.0       0.46      1.00      0.63        70
         1.0       0.00      0.00      0.00        83

    accuracy                           0.46       153
   macro avg       0.23      0.50      0.31       153
weighted avg       0.21      0.46      0.29       153

Confusion matrix
[[70  0]
 [83  0]]


/media/ilia/Vol3/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Как видим, модель всегда предсказывает один и тот же класс. Вероятно, слишком мало сэмплов для обучения.

Построим модель ResNet 50 инициализированную весами ImageNet и обучим ее.

In [9]:
resnet50 = tf.keras.applications.ResNet50(include_top=False, weights='imagenet')
resnet50.trainable = False

i = tf.keras.layers.Input(shape=shape, dtype=tf.uint8)
o = tf.cast(i, tf.float32)
o = tf.keras.applications.resnet50.preprocess_input(o)
o = resnet50(o)
o = tf.keras.layers.Flatten()(o)
o = tf.keras.layers.Dense(1, activation='sigmoid')(o)

rniw_model = tf.keras.Model(inputs=[i], outputs=[o])
rniw_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rniw_model.summary()
rniw_model.fit(train, validation_data=test, epochs=10);

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.cast_1 (TFOpLambda)       (None, 224, 224, 3)       0         
_________________________________________________________________
tf.__operators__.getitem_1 ( (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add_1 (TFOpLambda (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
flatten_3 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1003

Посчитаем accuracy на отложенной выборке.

In [10]:
score_and_show(rniw_model, x_val, y_val)

Accuracy: 0.5490196078431373

Classification Report
              precision    recall  f1-score   support

         0.0       0.57      0.06      0.10        70
         1.0       0.55      0.96      0.70        83

    accuracy                           0.55       153
   macro avg       0.56      0.51      0.40       153
weighted avg       0.56      0.55      0.43       153

Confusion matrix
[[ 4 66]
 [ 3 80]]


На модели с весами ImageNet заметно явное предсказание в пользу одиного из классов. Наличие весов помогло модели несколько лучше произвести классификацию. Тем не менее, результы плохие. Как и в предыдущем случае, проблема скорее всего в малом кол-ве сэмплов для обучения.